In [1]:
import mediapipe as mp
import cv2 as cv
import pandas as pd
import os
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Joint Determinations through an active feed
<img src="joints.png" style="height:300px" >

In [2]:
cap = cv.VideoCapture(0)

# Using the Pose model with specific confidence thresholds
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolouring the image
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False  # Prevents unnecessary caching

        # Make pose detection
        results = pose.process(image)

        # Recolor the image back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        try:
            # Get pose landmarks
            landmarks = results.pose_landmarks.landmark

            # Get normalized x and y coordinates for all landmarks
            height, width, _ = image.shape
            x_coords = [landmark.x * width for landmark in landmarks]
            y_coords = [landmark.y * height for landmark in landmarks]

            # Calculate bounding box
            x_min, x_max = int(min(x_coords)), int(max(x_coords))
            y_min, y_max = int(min(y_coords)), int(max(y_coords))

            # Draw the bounding box
            cv.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Calculate and print height and width of the bounding box
            box_width = x_max - x_min
            box_height = y_max - y_min
            if (box_height<box_width):
                print("\n---\nheight less than width!\n---\n")
            print(f"Bounding Box: Width={box_width}, Height={box_height}")

            # Extract RIGHT_WRIST and LEFT_WRIST positions
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

            # Print RIGHT_WRIST and LEFT_WRIST data if visible
            if right_wrist.visibility > 0.5 and left_wrist.visibility > 0.5:
                print(f"RIGHT_WRIST: x={right_wrist.x}, y={right_wrist.y}, z={right_wrist.z}, visibility={right_wrist.visibility}")
                print(f"LEFT_WRIST: x={left_wrist.x}, y={left_wrist.y}, z={left_wrist.z}, visibility={left_wrist.visibility}")

        except AttributeError:
            pass

        # Render pose landmarks on the image
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # Display the image
        cv.imshow('Mediapipe Feed', image)

        # Break the loop if 'q' is pressed
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv.destroyAllWindows()


Bounding Box: Width=434, Height=1285
Bounding Box: Width=446, Height=1262
Bounding Box: Width=459, Height=1252
Bounding Box: Width=439, Height=1256
Bounding Box: Width=422, Height=1259
Bounding Box: Width=419, Height=1267
Bounding Box: Width=430, Height=1283
Bounding Box: Width=454, Height=1292
Bounding Box: Width=466, Height=1291
Bounding Box: Width=474, Height=1290
Bounding Box: Width=488, Height=1289
Bounding Box: Width=485, Height=1289
Bounding Box: Width=472, Height=1291
Bounding Box: Width=466, Height=1286
Bounding Box: Width=461, Height=1283
Bounding Box: Width=464, Height=1285
Bounding Box: Width=470, Height=1291
Bounding Box: Width=472, Height=1293
Bounding Box: Width=471, Height=1292
Bounding Box: Width=470, Height=1292
Bounding Box: Width=474, Height=1291
Bounding Box: Width=479, Height=1291
Bounding Box: Width=481, Height=1291
Bounding Box: Width=479, Height=1295
Bounding Box: Width=461, Height=1293
Bounding Box: Width=455, Height=1293
Bounding Box: Width=456, Height=1292
B

# Reading through training data and marking coordinates into a csv file

>Whats happening here is simply a classification algorithm that mounts joints onto a person based on its visibility and then places a boundary box around the subject. Having done so the coordinates of these joints will be put into a CSV file alongside the image it was found in (the name of the image states if the person has fallen or not hence making it a trainable dataset

In [3]:
# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Path to the folder containing images
input_folder = "C:\\Users\\Ethan\\Documents\\Codes\\CM2603-Data_Science_Project-G20\\Fall_Detection\\fall_dataset\\images\\train"
output_csv = "image_train_pose_coordinates.csv"

# Initialize a DataFrame to store joint data
columns = ["Image", "Joint", "X", "Y", "Visibility", "Box_Width", "Box_Height"]
pose_data = []

# Using the Pose model
with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:  # Lowered thresholds
    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)

        # Check if it's an image file
        if not (image_name.endswith(".jpg") or image_name.endswith(".png") or image_name.endswith(".jpeg")):
            print(f"Skipping non-image file: {image_name}")
            continue

        # Read the image
        image = cv.imread(image_path)
        if image is None:
            print(f"Skipping invalid image file: {image_name}")
            continue

        # Resize image for consistency and faster processing (optional)
        image = cv.resize(image, (640, 480))

        # Process the image
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Check if landmarks are detected
        if results.pose_landmarks and results.pose_landmarks.landmark:
            print(f"Pose landmarks detected for: {image_name}")
            # Get pose landmarks
            landmarks = results.pose_landmarks.landmark

            # Get image dimensions
            height, width, _ = image.shape

            # Initialize bounding box variables
            x_coords = []
            y_coords = []

            # Extract and save landmark data
            for idx, landmark in enumerate(landmarks):
                x = landmark.x * width
                y = landmark.y * height
                visibility = landmark.visibility
                joint_name = mp_pose.PoseLandmark(idx).name
                pose_data.append([image_name, joint_name, x, y, visibility, 0, 0])  # Placeholder for box width/height

                # Collect x and y coordinates for bounding box calculation
                x_coords.append(x)
                y_coords.append(y)

            # Calculate bounding box (min/max coordinates)
            x_min = min(x_coords)
            x_max = max(x_coords)
            y_min = min(y_coords)
            y_max = max(y_coords)

            box_width = x_max - x_min
            box_height = y_max - y_min

            # Update the CSV data with bounding box dimensions
            for i in range(len(pose_data)):
                if pose_data[i][0] == image_name:
                    pose_data[i][5] = box_width
                    pose_data[i][6] = box_height

            # Draw landmarks on the image
            annotated_image = image.copy()
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )

            # Draw the bounding box around the pose
            cv.rectangle(annotated_image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)

            # Save the annotated image
            output_folder = "annotated_images"
            os.makedirs(output_folder, exist_ok=True)
            output_image_path = os.path.join(output_folder, f"annotated_{image_name}")
            cv.imwrite(output_image_path, annotated_image)
        else:
            print(f"No pose landmarks detected in: {image_name}")

# Save data to CSV
pose_df = pd.DataFrame(pose_data, columns=columns)
pose_df.to_csv(output_csv, index=False)
print(f"Pose coordinates saved to {output_csv}")

Pose landmarks detected for: fall001.jpg
No pose landmarks detected in: fall002.jpg
Pose landmarks detected for: fall003.jpg
Pose landmarks detected for: fall004.jpg
Pose landmarks detected for: fall005.jpg
Pose landmarks detected for: fall006.jpg
Pose landmarks detected for: fall007.jpg
No pose landmarks detected in: fall008.jpg
Pose landmarks detected for: fall009.jpg
Pose landmarks detected for: fall010.jpg
Pose landmarks detected for: fall011.jpg
Pose landmarks detected for: fall012.jpg
Pose landmarks detected for: fall013.jpg
Pose landmarks detected for: fall014.jpg
Pose landmarks detected for: fall015.jpg
Pose landmarks detected for: fall016.jpg
No pose landmarks detected in: fall017.jpg
Pose landmarks detected for: fall018.jpg
Pose landmarks detected for: fall019.jpg
Pose landmarks detected for: fall020.jpg
Pose landmarks detected for: fall021.jpg
Pose landmarks detected for: fall022.jpg
Pose landmarks detected for: fall023.jpg
Pose landmarks detected for: fall024.jpg
Pose landm

# Pre Processing the dataset

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the CSV file
csv_path = "image_train_pose_coordinates.csv"
df = pd.read_csv(csv_path)

# Extract the target label (fall or no_fall) from the image name
df["Label"] = df["Image"].apply(lambda x: 1 if x.lower().startswith("fall") else 0)

# Features and target
features = ["X", "Y", "Visibility", "Box_Width", "Box_Height"]
X = df[features]
y = df["Label"]

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Count the number of falls and non-falls
fall_counts = df["Label"].value_counts()

# Display the counts
print("Number of non-falls:", fall_counts[0])
print("Number of falls:", fall_counts[1])

Number of non-falls: 4686
Number of falls: 5676


# Training the model

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8099372889532079
              precision    recall  f1-score   support

           0       0.79      0.77      0.78       916
           1       0.82      0.84      0.83      1157

    accuracy                           0.81      2073
   macro avg       0.81      0.81      0.81      2073
weighted avg       0.81      0.81      0.81      2073


# Using an active feed

In [6]:
import cv2 as cv
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Preprocess frame
        image = cv.resize(frame, (640, 480))
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        # Detect pose landmarks
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            height, width, _ = image.shape
            landmarks = results.pose_landmarks.landmark

            # Extract features for prediction
            x_coords = [lm.x * width for lm in landmarks]
            y_coords = [lm.y * height for lm in landmarks]
            visibilities = [lm.visibility for lm in landmarks]

            # Calculate bounding box
            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)
            box_width = x_max - x_min
            box_height = y_max - y_min

            # Prepare input for the model
            features_input = np.array([np.mean(x_coords), np.mean(y_coords), np.mean(visibilities), box_width, box_height])
            features_input_scaled = scaler.transform([features_input])

            # Make prediction
            prediction = model.predict(features_input_scaled)[0]
            label = "Falling" if prediction == 1 else "Not Falling"

            # Annotate the frame
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )
            cv.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
            cv.putText(frame, label, (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame
        cv.imshow("Fall Detection", frame)

        # Break loop on 'q' key press
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

C:\Users\Ethan\.virtualenvs\Pose_Estimation\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Ethan\.virtualenvs\Pose_Estimation\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Ethan\.virtualenvs\Pose_Estimation\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Ethan\.virtualenvs\Pose_Estimation\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Ethan\.virtualenvs\Pose_Estimation\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but St